In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter
from functools import partial
from pathlib import Path

import awkward as ak
import bottleneck as bn
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/pLIB419.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Realign

In [ ]:
df = pl.read_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/dorado_0.4.0/uncompressed/prepared/consensus_spoa2/align2/combined.arrow"
)

In [ ]:
df.estimated_size(unit="mb")

In [ ]:
{col: df.get_column(col).estimated_size(unit="mb") for col in df.columns}

In [ ]:
path = df[0, "path_consensus"].to_list()
consensus_seq = df[0, "consensus_seq"]
cigar = df[0, "cg"]

In [ ]:
ref_seq = sgfa.assemble_seq_from_path(gfa, path)

In [ ]:
score, cigar = align.pairwise_align(
    consensus_seq,
    ref_seq,
    degenerate=True,
    gap_opening=12,
    gap_extension=3,
    match=2,
    mismatch=-1,
    parasail_algorithm="nw",
)
(score, cigar)

In [ ]:
score, cigar = align.pairwise_align(
    consensus_seq,
    ref_seq,
    degenerate=True,
    gap_opening=12,
    gap_extension=3,
    match=2,
    mismatch=-1,
)
(score, cigar)

In [ ]:
score, cigar = align.pairwise_align(consensus_seq, ref_seq, degenerate=True)
(score, cigar)

In [ ]:
def func(row, name_to_seq=None, **kwargs):
    path, seq = row
    ref_seq = sgfa.assemble_seq_from_path(name_to_seq, path)
    score, cigar = align.pairwise_align(seq, ref_seq, **kwargs)
    return score, cigar
    # return dict(a=score, b=cigar)
    # return dict(a=1, b=2)


name_to_seq = sgfa.gfa_name_mapping(gfa)
df.head(2).select(pl.col("path_consensus", "consensus_seq")).map_rows(
    partial(func, name_to_seq=name_to_seq),
    # return_dtype=pl.Struct(dict(a=pl.Int32, b=pl.Int32)),
    # return_dtype=pl.Struct(dict(score_realign=pl.Int32, cg_realign=pl.Utf8)),
).rename({"column_0": "score_realign", "column_1": "cg_realign"}).with_columns(
    pl.col("score_realign").cast(pl.Int32)
)

# Identify variants

In [ ]:
df2 = pl.read_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/dorado_0.4.0/uncompressed/prepared/consensus_spoa2/align2/combined_realigned.arrow"
)

In [ ]:
len(df2)

In [ ]:
df2.estimated_size(unit="mb")

In [ ]:
{col: df2.get_column(col).estimated_size(unit="mb") for col in df2.columns}

In [ ]:
df2[:10, "cg_realign"].to_list()

In [ ]:
idx = 3
seq = df2[idx, "consensus_seq"]
cigar = df2[idx, "cg_realign"]
path = df2[idx, "path_consensus"].to_list()

In [ ]:
cigar

In [ ]:
"".join(path)

In [ ]:
cigar_d = align.decode_cigar(cigar)

In [ ]:
cigar_d

In [ ]:
name_to_seq = sgfa.gfa_name_mapping(gfa)

In [ ]:
# if CIGAR has insertions on a segment boundary, where should they go??
# (should be insensitive to rc-ing??)
# if we solve this, can we fix flanking problem?

# keep start/end as variables, record them all at once?
# use them to allow returning sequences even if return_indices=False

# trim insertions from each flank? or associate those with first/last segment? option to do either?
# test query_seq/ref_seq
# hhh

# code duplication

scigar.cut_cigar(cigar_d, path, name_to_seq, sequence=seq, cigar_as_string=False)

In [ ]:
%timeit cut_cigar(cigar_d, path, name_to_seq)

In [ ]:
%lprun -f cut_cigar cut_cigar(cigar_d, path, name_to_seq)